In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from time import time

In [2]:
df = pd.read_csv('datasets/SatisfactionClients/cdiscount2.csv')
df_test = pd.read_csv('datasets/SatisfactionClients/amazon_test1.csv')
df.sample(3)

,note,commentaire
64829,5,très bon site commande souvent
52160,5,cdiscount toujours rendez merci
11508,1,propose fois sans frais écrit gros tout site u...


In [3]:
df.isna().sum()

note            0
commentaire    54
dtype: int64

In [4]:
df[df['commentaire'].isna()==True]

,note,commentaire
156,5,NaN
364,4,NaN
473,5,NaN
476,5,NaN
1459,5,NaN
2040,3,NaN
2144,5,NaN
2293,5,NaN
2749,5,NaN
2920,4,NaN


In [5]:
df = df[df['commentaire'].isna()==False]

In [6]:
# séparation de la variable cible et des variables explicatives
X = df['commentaire']
y = df['note']

In [7]:
# séparation du jeu de données en un dataset d'entrainement et un dataset de test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)

In [8]:
# conversion des chaines de caratères en tokens numériques
# on ne prendra que les éléments répétés au moins 3 fois avec min_df=3
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features=5000, min_df=3)
X_train = vectorizer.fit_transform(X_train).todense()
X_test = vectorizer.transform(X_test).todense()

In [9]:
np.save('datasets/SatisfactionClientsvector_matrix', X_train)

In [10]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report

clf = GradientBoostingClassifier(n_estimators=25)

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(clf.score(X_test, y_test), end='\n\n')

cr = classification_report(y_test, y_pred)
print(cr)

cm = pd.crosstab(y_test, y_pred, rownames=['données réelles'], colnames=['predictions'])
display(cm)

# vérification des résultats sur un jeu de test externe (100 commentaires amazon également répartis entre les étoiles)
df_test_token = vectorizer.transform(df_test['commentaire']).todense()
y_predict_test = clf.predict(df_test_token)

cm_test = pd.crosstab(df_test['note'], y_predict_test, rownames=['données réelles'], colnames=['predictions'])
display(cm_test)

cm_test = pd.crosstab(df_test['note'], y_predict_test, rownames=['données réelles'], colnames=['predictions'], normalize=0)
display(cm_test)

for i in cm_test.index:
    for j in cm_test.columns :
        if (i==j and cm_test.loc[i, j] > 0.5) :
            print("les prédictions correctes sont supérieures à 50% pour {} étoile(s) avec {} %".format(i, cm_test.loc[i, j]*100))

C:\Users\t.fourtouill\Anaconda3\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
C:\Users\t.fourtouill\Anaconda3\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
C:\Users\t.fourtouill\Anaconda3\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


0.6371387967787778

              precision    recall  f1-score   support

           1       0.81      0.48      0.60      2037
           2       0.14      0.00      0.00       464
           3       0.44      0.01      0.02      1150
           4       0.49      0.02      0.04      3475
           5       0.63      0.99      0.77      9762

    accuracy                           0.64     16888
   macro avg       0.50      0.30      0.29     16888
weighted avg       0.59      0.64      0.53     16888



predictions,1,2,3,4,5
données réelles,,,,,
1,969,4,6,8,1050
2,86,1,7,8,362
3,80,2,12,34,1022
4,36,0,2,75,3362
5,30,0,0,29,9703


C:\Users\t.fourtouill\Anaconda3\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


predictions,1,4,5
données réelles,,,
1,10,0,10
2,6,0,14
3,5,5,10
4,3,2,15
5,0,0,20


predictions,1,4,5
données réelles,,,
1,0.50,0.00,0.50
2,0.30,0.00,0.70
3,0.25,0.25,0.50
4,0.15,0.10,0.75
5,0.00,0.00,1.00


les prédictions correctes sont supérieures à 50% pour 5 étoile(s) avec 100.0 %


In [11]:
from joblib import dump
dump(clf, 'datasets/SatisfactionClients/model_gbc.joblib')

['datasets/SatisfactionClients/model_dtc.joblib']

### Evaluation sur 2 sentiments : négatif et positif

* Le DataSet cdiscount a été retraité en renommant, les notes 1 et 2 en 0, les notes 4 et 5 en 1 et en supprimant les notes 3
* Le but étant d'isoler de manière moins complexe les sentiments négatifs et les sentiments positifs et ainsi améliorer la qualité du résultat prédit

In [12]:
df_0_1 = pd.read_csv('datasets/SatisfactionClients/cdiscount_0_1.csv')

In [13]:
df_0_1.isna().sum()

note            0
commentaire    46
dtype: int64

In [14]:
df_0_1 = df_0_1[df_0_1['commentaire'].isna()==False]

In [15]:
# séparation de la variable cible et des variables explicatives
X1 = df_0_1['commentaire']
y1 = df_0_1['note']

In [16]:
# séparation du jeu de données en un dataset d'entrainement et un dataset de test
from sklearn.model_selection import train_test_split
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1, test_size=0.2, shuffle=True)

In [17]:
X_train1 = vectorizer.transform(X_train1).todense()
X_test1 = vectorizer.transform(X_test1).todense()

In [18]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report

clf_0_1 = GradientBoostingClassifier(n_estimators=25)

clf_0_1.fit(X_train1, y_train1)

y_pred_0_1 = clf_0_1.predict(X_test1)

print(clf_0_1.score(X_test1, y_test1), end='\n\n')

cr = classification_report(y_test1, y_pred_0_1)
print(cr)

cm = pd.crosstab(y_test1, y_pred_0_1, rownames=['données réelles'], colnames=['predictions'])
display(cm)

# vérification des résultats sur un jeu de test externe (100 commentaires amazon également répartis entre les étoiles)
df_test_token = vectorizer.transform(df_test['commentaire']).todense()
y_predict_test = clf_0_1.predict(df_test_token)

cm_test = pd.crosstab(df_test['note'], y_predict_test, rownames=['données réelles'], colnames=['predictions'])
display(cm_test)

cm_test2 = pd.crosstab(df_test['note'], y_predict_test, rownames=['données réelles'], colnames=['predictions'], normalize=0)
display(cm_test2)

for i in cm_test2.index:
    for j in cm_test2.columns :
        if (i==j and cm_test2.loc[i, j] > 0.5) :
            print("les prédictions correctes sont supérieures à 50% pour {} étoile(s) avec {} %".format(i, cm_test2.loc[i, j]*100))

C:\Users\t.fourtouill\Anaconda3\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
C:\Users\t.fourtouill\Anaconda3\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
C:\Users\t.fourtouill\Anaconda3\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


0.9004883617682501

              precision    recall  f1-score   support

           0       0.94      0.42      0.58      2574
           1       0.90      0.99      0.94     13193

    accuracy                           0.90     15767
   macro avg       0.92      0.71      0.76     15767
weighted avg       0.90      0.90      0.88     15767



predictions,0,1
données réelles,,
0,1071,1503
1,66,13127


C:\Users\t.fourtouill\Anaconda3\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


predictions,0,1
données réelles,,
1,9,11
2,5,15
3,5,15
4,4,16
5,2,18


predictions,0,1
données réelles,,
1,0.45,0.55
2,0.25,0.75
3,0.25,0.75
4,0.20,0.80
5,0.10,0.90


les prédictions correctes sont supérieures à 50% pour 1 étoile(s) avec 55.00000000000001 %


In [19]:
from joblib import dump
dump(clf_0_1, 'datasets/SatisfactionClients/model_gbc_0_1.joblib')

['datasets/SatisfactionClients/model_dtc_0_1.joblib']